In [1]:
# 패키지 불러오기
import pandas as pd # 데이터 처리
import numpy as np #연산
import matplotlib #시각화
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.api as s
import seaborn as sns
import math
import matplotlib.font_manager as fm
import warnings
import graphviz
warnings.filterwarnings('ignore')
import statsmodels.api as sm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error as mse

from sklearn.model_selection import train_test_split # 데이터 분할 : train, test
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor #예측/회귀
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from datetime import datetime
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest
from sklearn.feature_selection import RFE
from statsmodels.api import qqplot, add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import r2_score
from scipy.stats import mannwhitneyu
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from datetime import date
from sklearn.tree import export_graphviz
from sklearn.metrics import confusion_matrix, classification_report

path='/usr/share/fonts/opentype/noto/NotoSansCJK-Bold.ttc'
fontprop=fm.FontProperties(fname=path, size=18).get_name()
matplotlib.rc('font',family = fontprop)
matplotlib.rc('axes', unicode_minus = False)

%matplotlib inline

pd.set_option('display.max_columns', None)
pd.options.display.max_rows = 400

from sklearn.neighbors import KNeighborsClassifier

In [2]:
df1 = pd.read_csv("01_sales_data2.csv")
df2 = pd.read_csv("02_customer_data.csv")
df3 = pd.read_csv("03_stock_data.csv")

df = pd.merge(df1,df2,on = ["customer_id"])
df = pd.merge(df, df3, on=["product_name"])

In [3]:
df_cleaned = df.copy(deep = True)
df_cleaned['QuantityCanceled'] = 0
entry_to_remove = []
doubtfull_entry = []
for index, col in  df.iterrows(): #interrows 객채로 변환
    # 주문 수량이 0보다 크거나 할인의 경우
    if (col['quantity'] > 0):
        continue
            # 수량이 +이면 그냥 대입
        # 수량이 -이면 아래것을 돌림
    df_test = df[(df['customer_id'] == col['customer_id']) &
                         (df['product_code']  == col['product_code']) &
                         (df['date'] <= col['date']) &
                         (df['quantity']   > 0)].copy()
    # 주문일이 더 빠르고 수량이 0보다 큰 경우가 1개도 없는 경우
    if (df_test.shape[0] == 0):
        doubtfull_entry.append(index)
    # 주문일이 더 빠르고 수량이 0보다 큰 경우가 1개 있는 경우 (수량만큼 마이너스 값이 들어감)
    elif (df_test.shape[0] == 1):
        index_order = df_test.index[0]
        # 취소된 수량에 대한 컬럼을 새로 생성한다.
        df_cleaned.loc[index_order, 'QuantityCanceled'] = -col['quantity']
        entry_to_remove.append(index)
    # 주문일이 더 빠르고 수량이 0보다 큰 경우가 2개 이상인 경우
    elif (df_test.shape[0] > 1):
        df_test.sort_index(axis=0 ,ascending=False, inplace = True)
        # 부분취소를 한 경우
        for ind, val in df_test.iterrows():
            if val['quantity'] < -col['quantity']: continue
            df_cleaned.loc[ind, 'QuantityCanceled'] = -col['quantity']
            entry_to_remove.append(index)
            break 
            
# entry_to_remove : 없애줘야할것 = 마이너스 값인데, 주문일이 더 빠르고 수량이 +인 것이 이전에 있는 인덱스가 경우
# doubtfull_entry : 의심되는것 ex) 마이너스 값인데 전에 구매한 것이 없는거
# 둘다 인덱스만 넣어줬다는 거 주의!!!!
# 원래 df_cleaned -> df1

In [4]:
print("entry_to_remove: {}".format(len(entry_to_remove)))
# 삭제 데이터 = 취소했는데 전에 산 수량보다 큰 수량으로 취소한 데이터
print("doubtfull_entry: {}".format(len(doubtfull_entry)))
# 이상한 데이터 = 취소했는데 시킨게 없는거

entry_to_remove: 4687
doubtfull_entry: 1129


In [5]:
df_cleaned.drop(entry_to_remove, axis = 0, inplace = True)
df_cleaned.drop(doubtfull_entry, axis = 0, inplace = True)
# 둘다 지움 !
# 이제 df1에는 환불한것들이랑 의심되는 것들 없음, df1 = 진짜 주문 수량
# 중요한건 이 데이터 프레임은 환불횟수만 쓸 수 있음

In [6]:
df_cleaned.sort_values(by = "quantity")
df_cleaned.drop("quantity" > 10000, axis = 0, inplace = True)

TypeError: '>' not supported between instances of 'str' and 'int'

# 이제 df_cleaned 를 사용하면 됩니다.
# entry_to_remove은 환불한 자료입니다. doubtfull_entry는 주문 안했는데 주문한 자료입니다.인덱스로 표현되어 있으니까 iloc 써서 확인하면 됩니다.
# 이거는 하루만에 환불한 것도 삭제된 데이터 입니다.
# 질문사항이 있다면 김성언 대리님께 질문해주세요.